In [145]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json
import urllib.request

### DATOS DE CONECTIVIDAD A INTERNET EN ARGENTINA

*Fuente: https://datosabiertos.enacom.gob.ar/dashboards/20000/acceso-a-internet/*

In [146]:
### Leyendo Request a la API
with urllib.request.urlopen('https://api.datosabiertos.enacom.gob.ar/api/v2/resources.json/?auth_key=ELJioxD1pDg7gdxvQfugkirauHiVrOSfliF5ElWo') as data2:
    data2 = json.load(data2)

### La API devuelve una lista con los recursos disponibles
recursos = list()
for item in data2:
    recursos.append({'Descripcion': item['description'], 'Titulo': item['title'], 'Categoria': item['category_name'], 'GUID': item['guid'], 'Tipo': item['type']})
    
recursos = pd.DataFrame(recursos)
datasets = recursos.query('Tipo == "dt"').query('Categoria == "Acceso a Internet"') # Se accede a los recursos que son Datasets de la categoría 'Acceso a internet'

### Se accede a cada recurso y se busca su origen
bucket_s3 = list()
for i, dataset in datasets.iterrows():
    with urllib.request.urlopen(f'https://api.datosabiertos.enacom.gob.ar/api/v2/datasets/{dataset["GUID"]}.json/?auth_key=ELJioxD1pDg7gdxvQfugkirauHiVrOSfliF5ElWo') as data:
        data = json.load(data)
    bucket_s3.append({'Titulo': data['title'], 'Descripcion': data['description'], 'GUID': data['guid'], 'URL': data['download_url']})

In [147]:
mapa_conexion = pd.read_excel(bucket_s3[0]['URL'])
penetracion = pd.read_excel(bucket_s3[1]['URL'])

accesos_tecnologia_localidad = pd.read_excel(bucket_s3[2]['URL'])
accesos_tecnologia_localidad = accesos_tecnologia_localidad.drop(columns=['Total general']).melt(id_vars=['Provincia', 'Partido', 'Localidad', 'Link Indec'], var_name='Tipo Conexion', value_name='Cantidad')

accesos_velocidad_provincia = pd.read_excel(bucket_s3[3]['URL'])
accesos_velocidad_provincia = accesos_velocidad_provincia.melt(id_vars=['Provincia', 'Partido', 'Localidad', 'Link Indec'], var_name='Velocidad', value_name='Valor').fillna(0)

velocidad_bajada = pd.read_excel(bucket_s3[4]['URL'])

normalizacion = {'Ciudad Autónoma de Buenos Aires': 'CABA','Capital Federal': 'CABA', 'Buenos Aires': 'BUENOS AIRES', '24 Partidos del Gran Buenos Aires': 'BUENOS AIRES', 
                 'Resto de partidos de la Provincia\nde Buenos Aires': 'BUENOS AIRES', 'Catamarca': 'CATAMARCA', 'Chaco': 'CHACO', 
                 'Chubut': 'CHUBUT','Córdoba': 'CORDOBA','Corrientes': 'CORRIENTES','Entre Ríos': 'ENTRE RIOS', 'Formosa': 'FORMOSA',
                 'Jujuy': 'JUJUY', 'La Pampa': 'LA PAMPA', 'La Rioja': 'LA RIOJA', 'Mendoza': 'MENDOZA', 'Misiones': 'MISIONES',
                 'Neuquén': 'NEUQUEN', 'Río Negro': 'RIO NEGRO', 'Salta': 'SALTA', 'San Juan': 'SAN JUAN', 'San Luis': 'SAN LUIS',
                 'Santa Cruz': 'SANTA CRUZ', 'Santa Fe': 'SANTA FE', 'Santiago del Estero': 'SANTIAGO DEL ESTERO', 'Santiago Del Estero': 'SANTIAGO DEL ESTERO',
                 'Tierra del Fuego, Antártida e Islas\ndel Atlántico Sur (2)': 'TIERRA DEL FUEGO', 'Tierra Del Fuego': 'TIERRA DEL FUEGO',
                 'Tucumán': 'TUCUMAN'}

velocidad_bajada['Provincia'] = velocidad_bajada['Provincia'].replace(normalizacion)

accesos_bandas = pd.read_excel(bucket_s3[5]['URL'])
accesos_bandas['Provincia'] = accesos_bandas['Provincia'].replace(normalizacion)

accesos_tipo_conexion = pd.read_excel(bucket_s3[6]['URL'])
accesos_tipo_conexion = accesos_tipo_conexion.drop(columns='Total').melt(id_vars=['Año', 'Trimestre', 'Periodo'], var_name='Tipo Conexion', value_name='Cantidad')

accesos_rango_velocidad = pd.read_excel(bucket_s3[7]['URL'])
accesos_rango_velocidad = accesos_rango_velocidad.drop(columns='Total').melt(id_vars=['Año', 'Trimestre'])

accesos_velocidad_bajada = pd.read_excel(bucket_s3[8]['URL'])
accesos_velocidad_bajada.replace(normalizacion).melt(id_vars = ['Año', 'Trimestre', 'Provincia']).fillna(0)

ingresos_operador = pd.read_excel(bucket_s3[9]['URL'])

datos_macroeconomicos = pd.read_excel(bucket_s3[11]['URL'])

provincias = accesos_bandas['Provincia'].unique()
anios = velocidad_bajada['Año'].unique()
trimestres = velocidad_bajada['Trimestre'].sort_values().unique()


# DATOS DE POBLACIÓN EN ARGENTICA POR PROVINCIA
### CENSO 2022

*Fuente https://censo.gob.ar/index.php/datos_provisionales/*

In [77]:
data_poblacion = pd.read_excel('https://www.indec.gob.ar/ftp/cuadros/poblacion/cnphv2022_resultados_provisionales.xlsx', sheet_name='Cuadro 1')
data_poblacion.columns = [data_poblacion.iloc[1].values]
data_poblacion.drop([0,1,2,5,6,29,30,31,32], inplace=True)
data_poblacion.reset_index(drop=True, inplace=True)

normalizacion = {'Ciudad Autónoma de Buenos Aires': 'CABA', 'Buenos Aires': 'BUENOS AIRES', '24 Partidos del Gran Buenos Aires': 'BUENOS AIRES', 
                 'Resto de partidos de la Provincia\nde Buenos Aires': 'BUENOS AIRES', 'Catamarca': 'CATAMARCA', 'Chaco': 'CHACO', 
                 'Chubut': 'CHUBUT','Córdoba': 'CORDOBA','Corrientes': 'CORRIENTES','Entre Ríos': 'ENTRE RIOS', 'Formosa': 'FORMOSA',
                 'Jujuy': 'JUJUY', 'La Pampa': 'LA PAMPA', 'La Rioja': 'LA RIOJA', 'Mendoza': 'MENDOZA', 'Misiones': 'MISIONES',
                 'Neuquén': 'NEUQUEN', 'Río Negro': 'RIO NEGRO', 'Salta': 'SALTA', 'San Juan': 'SAN JUAN', 'San Luis': 'SAN LUIS',
                 'Santa Cruz': 'SANTA CRUZ', 'Santa Fe': 'SANTA FE', 'Santiago del Estero': 'SANTIAGO DEL ESTERO', 
                 'Tierra del Fuego, Antártida e Islas\ndel Atlántico Sur (2)': 'TIERRA DEL FUEGO', 'Tucumán': 'TUCUMAN'}

data_poblacion['Jurisdicción'] = data_poblacion['Jurisdicción'].replace(normalizacion)
data_poblacion.drop(columns=['Población en situación de calle (vía pública)'], inplace=True)